In [1]:
from __init__ import subject_number, exp, threshold_move, threshold_time, aruduino_no, columns, path_data
import sys
sys.dont_write_bytecode = True # __pycache__を消す

In [5]:
####################################
# データ収集
# エンターキーでデータ収集を停止．
# Arduino IDEが動作している場合，アクセス拒否されるので，Arduino IDEを閉じる
# エンターキーで停止しなかった場合，アクセス拒否されるので，カーネルを再起動する
####################################

import serial
import keyboard
import pandas as pd

df = [] # 1回分の動作データを格納
dfs = [] # dfを格納

ser = serial.Serial(aruduino_no, 115200, timeout=None) # arduinoからデータをとってくる

while True:
    if keyboard.is_pressed("enter"): # エンターキーで終了
        break
    
    line = ser.readline() # ここで一行データを取得するがbyte型
    stripped_str = str(line, 'utf-8').strip() # byte型を文字列に変換して前後の空白改行除去
    stripped_str = stripped_str.split(',') # ,文字で区切る
    
    data = []
    for item in stripped_str: # float型に変換
        try:
            data.append(float(item))
        except ValueError:
            break
    
    # 動作しているときの処理
    if sum(abs(x) for x in data) >= threshold_move: # sum(abs(x) for x in data) が3.5以上なら動いたってしてよさそう．しきい値は要調整
        df.append(data)
    elif df: # 1回の動作が終了したときの処理
        df = pd.DataFrame(df, columns=columns) # 統計量を計算しやすいようにdataframeに変換
        dfs.append(df) # 動作をまとめて格納
        print(len(dfs))
        df = df.apply(lambda x : x.tolist(), axis=1).tolist() # appendで高速に格納するためlistに戻す
        df.clear() # 中身を空っぽに
    else: # 動いてないときの処理
        pass
    
ser.close() # arduinoとの通信を停止

In [3]:
####################################
# 特徴量抽出
####################################

averages = [] # 平均を格納するためのリスト
variances = [] # 分散を格納するための空のリスト
std_deviations = [] # 標準偏差を格納するための空のリスト
kurtosis_values = [] # 尖度を格納するための空のリスト
skewness_values = [] # 歪度を格納するための空のリスト
element_counts = [] # 要素数を格納するための空のリスト

for idx, df in enumerate(dfs):
    # 列ごとの特徴量を計算しリストに追加
    column_averages = df.apply(pd.to_numeric, errors='coerce', axis=0).mean(axis=0)
    averages.append(column_averages)
    column_variances = df.apply(pd.to_numeric, errors='coerce', axis=0).var(axis=0)
    variances.append(column_variances)
    column_std = df.apply(pd.to_numeric, errors='coerce', axis=0).std(axis=0)
    std_deviations.append(column_std)
    column_kurtosis = df.apply(pd.to_numeric, errors='coerce', axis=0).kurtosis(axis=0) 
    kurtosis_values.append(column_kurtosis)
    column_skewness = df.apply(pd.to_numeric, errors='coerce', axis=0).skew(axis=0)
    skewness_values.append(column_skewness)
    column_element_counts = df.apply(pd.to_numeric, errors='coerce', axis=0).count(axis=0)
    element_counts.append(column_element_counts)

# データフレームに変換    
averages_df = pd.DataFrame(averages)
variances_df = pd.DataFrame(variances)
std_deviations_df = pd.DataFrame(std_deviations)
kurtosis_df = pd.DataFrame(kurtosis_values)
skewness_df = pd.DataFrame(skewness_values)
times_df = pd.DataFrame(element_counts)

times_df = times_df[['ax']] # 1列目の値でDataFrameを分割

# ヘッダー行の文字の語尾に特徴名を追加
new_columns = [col + "_averages" for col in averages_df.columns]
averages_df.columns = new_columns
new_columns = [col + "_variances" for col in variances_df.columns]
variances_df.columns = new_columns
new_columns = [col + "_std_deviations" for col in std_deviations_df.columns]
std_deviations_df.columns = new_columns
new_columns = [col + "_kurtosis" for col in kurtosis_df.columns]
kurtosis_df.columns = new_columns
new_columns = [col + "_skewness" for col in skewness_df.columns]
skewness_df.columns = new_columns
new_columns = ["times" for col in times_df.columns]
times_df.columns = new_columns

# リストの要素数だけ1を持つDataFrameを作成
class_df = pd.DataFrame([subject_number] * len(dfs), columns=['class'])

# 各特徴が格納されたdataframeを連結
concatenated_df = class_df.join([averages_df, variances_df, std_deviations_df, kurtosis_df, skewness_df, times_df])

# とても短い動作を除去
df_filtered = concatenated_df[concatenated_df['times'] > threshold_time]

df = df_filtered
print(df.head())
df.to_csv(path_data + '/out_'+ str(subject_number) + '.csv', mode = 'a', header=exp, index=False)

    class  ax_averages  ay_averages  az_averages  gx_averages  gy_averages  \
1       1     0.268462    -0.783846    -0.101538    -3.208462   -12.363846   
5       1     0.383684    -0.840000    -0.132632     0.687895    -1.908421   
10      1     0.336667    -0.768571    -0.123333     1.708095    -1.569524   
12      1     0.409524    -0.787619    -0.119524    -0.051905     2.067619   
16      1     0.336667    -0.827619    -0.118095    -0.142381     1.606667   

    gz_averages  mx_averages  my_averages  mz_averages  ...  ax_skewness  \
1    108.088462    -0.148462     0.614615    -0.097692  ...     0.814294   
5     14.275789    -0.140000     0.621579    -0.087368  ...     0.193334   
10     0.929048    -0.135238     0.624286    -0.086667  ...     0.526633   
12    11.618095    -0.130476     0.629048    -0.084762  ...     0.034726   
16     6.753333    -0.145714     0.620476    -0.085714  ...     0.434553   

    ay_skewness  az_skewness  gx_skewness  gy_skewness  gz_skewness  \
1  